In [21]:
from urllib import request
from bs4 import BeautifulSoup as bf
import pandas as pd
import re
import time
import sqlite3

url="http://movie.douban.com/top250?start="
savepath=r"C:\Users\fh282\Desktop\film.xlsx"

title=re.compile(r'<span class="title">(.*?)</span>')
rating=re.compile(r'<span class="rating_num" property="v:average">(.*?)</span>')
review=re.compile(r'<span>(.*?)人评价</span>')
inq=re.compile(r'<span class="inq">(.*?)</span>')
actor=re.compile(r'<div class="bd"><p class="">(.*?)</p><div class="star">')

def geturllist(url,times):
    urllist=[]
    for i in range(times):
        myurl=url+str(i*25)
        urllist.append(myurl)
    return urllist

def getdata(urllist):
    header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36"}
    datalist=[]
    for url in urllist:
        req=request.Request(url,headers=header)
        response=request.urlopen(req)
        soup=bf(response,"html.parser")
        for item in soup.find_all("div",class_="item"):
            item=str(item)
            data=[]
            mytitle=re.findall(title,item)
            if len(mytitle)==2:
                data.append(mytitle[0])
                otitle=mytitle[1].replace("/","").replace("\xa0","")
                data.append(otitle)
            else:
                data.append(mytitle[0])
                data.append(" ")
            myrating=re.findall(rating,item)[0]
            data.append(myrating)
            myreview=re.findall(review,item)[0]
            data.append(myreview)
            myinq=re.findall(inq,item)
            if len(myinq)==1:
                data.append(myinq)
            else:
                data.append(" ")
            datalist.append(data)
        time.sleep(1)
    return datalist

def savedata(datalist,savepath):
    df=pd.DataFrame(datalist)
    df.columns=["影片名称","外文名","评分","评价人数","介绍"]
    df.to_excel(savepath)
    
def savetosql(datalist):
    con=sqlite3.connect(r"./douban.db")
    cur=con.cursor()
    sql="create table movies3(name varchar(45) primary key not null,fname varchar(45) not null,score varchar(45) not null,num varchar(45) not null,inq varchar(45) not null)"
    cur.execute(sql)
    for data in datalist:
        sqlv="insert into movies3 values(str(data[0]),str(data[1]),str(data[2]),str(data[3]),str(data[4]))"
        cur.execute(sqlv)
    con.commit()
    cur.close()
    vcon.close()
    
def main():
    #获取url列表
    urllist = geturllist(url,1)
    #获取data
    datalist=getdata(urllist)
    #存储数据至excel
#     savedata(datalist,savepath)
    #存储数据至数据库：sqlit3
    savetosql(datalist)
    
if __name__=="__main__":
    main()

OperationalError: table movies3 already exists